In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#parameters
classes = 12
image_size = 1000
batch_size= 32
train_total = 3809
validation_total = 941

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)

In [4]:
model_mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 1000, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 500, 500, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 500, 500, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 500, 500, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 500, 500, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 500, 500, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 500, 500, 16)      0         
__________

In [5]:
x = model_mobilenet.output
x = MaxPooling2D((4,4), strides=(4,4))(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 1000, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 500, 500, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 500, 500, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 500, 500, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 500, 500, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 500, 500, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 500, 500, 16)      0         
__________

In [8]:
model_mobilenet_2 = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)

In [9]:
x = model_mobilenet_2.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [10]:
model_2 = Model(inputs=model_mobilenet_2.input, outputs=x)

In [11]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 1000, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 500, 500, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 500, 500, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 500, 500, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 500, 500, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 500, 500, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 500, 500, 16)      0         
__________

In [12]:
print(model.layers[1].trainable)

True


In [13]:
model_2.load_weights('mobilenet_from_scratch_save_best_image_increase_1000.hdf5')

In [14]:
for i in range(len(model_mobilenet.layers)):
    layer_1 = model.layers[i]
    layer_2 = model_2.layers[i]
    layer_1.set_weights(layer_2.get_weights())

In [15]:
for layer in model_mobilenet.layers:
    layer.trainable = False

In [16]:
model.layers[-1].trainable

True

In [17]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 3809 images belonging to 12 classes.
Found 941 images belonging to 12 classes.


In [19]:
class_weights_custom = {}
for i in train_generator.class_indices:
    if(i=='Black-grass' or i=='Loose Silky-bent'):
        class_weights_custom[i]=4
    else:
        class_weights_custom[i]=1

In [20]:
class_weights_custom

{'Black-grass': 4,
 'Charlock': 1,
 'Cleavers': 1,
 'Common Chickweed': 1,
 'Common wheat': 1,
 'Fat Hen': 1,
 'Loose Silky-bent': 4,
 'Maize': 1,
 'Scentless Mayweed': 1,
 'Shepherds Purse': 1,
 'Small-flowered Cranesbill': 1,
 'Sugar beet': 1}

In [21]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [ ]:
callbacks = [ModelCheckpoint(filepath='mobilenet_fixed_bottom_save_best_image_increase_1000.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./14-metrics_increase.csv')]

In [ ]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=callbacks,
                    class_weight=[4,1,1,1,1,1,4,1,1,1,1,1])

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.81379, saving model to mobilenet_fixed_bottom_save_best_image_increase_1000.hdf5
 - 762s - loss: 9.2307 - acc: 0.2610 - val_loss: 0.8138 - val_acc: 0.7758
Epoch 2/100
Epoch 00002: val_loss improved from 0.81379 to 0.39344, saving model to mobilenet_fixed_bottom_save_best_image_increase_1000.hdf5
 - 678s - loss: 3.5750 - acc: 0.4685 - val_loss: 0.3934 - val_acc: 0.8502
Epoch 3/100
Epoch 00003: val_loss improved from 0.39344 to 0.22450, saving model to mobilenet_fixed_bottom_save_best_image_increase_1000.hdf5
 - 669s - loss: 2.0052 - acc: 0.6179 - val_loss: 0.2245 - val_acc: 0.9277
Epoch 4/100
Epoch 00004: val_loss improved from 0.22450 to 0.13990, saving model to mobilenet_fixed_bottom_save_best_image_increase_1000.hdf5
 - 669s - loss: 1.4966 - acc: 0.6859 - val_loss: 0.1399 - val_acc: 0.9543
Epoch 5/100
Epoch 00005: val_loss improved from 0.13990 to 0.12381, saving model to mobilenet_fixed_bottom_save_best_image_increase_1000.hdf

In [33]:
model.load_weights('mobilenet_fixed_bottom_save_best_image_increase_1000.hdf5')

In [22]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[7.1129420752479726, 0.10839532412327312]

In [17]:
# validation predictions

In [18]:
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 941 images belonging to 12 classes.


In [19]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))

In [20]:
predictions_valid = np.argmax(predictions_valid, axis=1)


In [21]:
true_positive = np.zeros(12)
false_positive = np.zeros(12)
true_negative = np.zeros(12)
false_negative = np.zeros(12)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [22]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)

true_positive
[  44.   82.   63.  109.   41.   93.  130.   43.  107.   45.  107.   65.]
false_positive
[ 4.  1.  1.  0.  0.  0.  5.  0.  1.  0.  0.  0.]
false_negative
[ 4.  0.  1.  0.  0.  1.  4.  1.  0.  1.  0.  0.]
Total
[  48.   82.   64.  109.   41.   94.  134.   44.  107.   46.  107.   65.]


In [ ]:
# test submission

In [34]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [35]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [36]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_from_scratch.csv', encoding="utf8", index=False)

In [37]:
from IPython.display import FileLink
FileLink('submission_mobilenet_from_scratch.csv')

/home/janardhan/plant_seeding/submission_mobilenet_from_scratch.csv

In [ ]:
# PB LB: 0.96977
from IPython.display import FileLink
FileLink('submission_mobilenet_two_models_v3.csv')